# Oxford 102 Flowers

In [ ]:
!pip install wandb==0.16

## Import packages, set configuration

In [ ]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import random
import json
from tensorflow.keras import layers
import tensorflow as tf
from PIL import Image
import tensorflow.keras.backend as K
import tensorflow_probability as tfp
from scipy.stats import beta
import pickle 

class CFG:
    # Directories
    DIR_NAME = '/kaggle/input/pytorch-challange-flower-dataset/dataset/'
    TRAIN_DIR = os.path.join(DIR_NAME, 'train')
    VAL_DIR = os.path.join(DIR_NAME, 'valid')
    TEST_DIR = os.path.join(DIR_NAME, 'test')
    
    # Images
    CNN_INPUT_HEIGHT = 224
    CNN_INPUT_WIDTH = 224
    CNN_INPUT_SIZE = (CNN_INPUT_HEIGHT, CNN_INPUT_WIDTH)
    CNN_INPUT_CHANNELS = 3
    CNN_INPUT_SHAPE = (CNN_INPUT_HEIGHT, CNN_INPUT_WIDTH, CNN_INPUT_CHANNELS)
    
    # Categories
    CATEGORIES_JSON_PATH = '/kaggle/input/pytorch-challange-flower-dataset/cat_to_name.json'
    
    with open(CATEGORIES_JSON_PATH) as f:
        CATEGORIES = json.load(f)
    CATEGORIES = {int(k): v for k,v in CATEGORIES.items()}
    
    # Dataset
    BATCH_SIZE = 32
    TRAIN_SET_SIZE = 6552
    
    # Dataset Augmentation
    AUGMENT = True
    TRADITIONAL_AUG_PROB = 0.4
    
    MIXUP_PROB = 0 # 0.4
    MIXUP_ALPHA = 0.4

    CUTMIX_PROB = 0 # 0.4
    CUTMIX_ALPHA = 1
    
    # Seeding
    SEED = 32
    
    # Model settings
    NETWORK_NAME = "ResNet"
    optimizer = "Adam"
    loss ="CCE"
    lr = 1e-5 # For Optimizer
    label_smoothing=0.0 # For loss (CCE)
    
    # Training
    NUM_SPLITS = 5
#     SELECTED_FOLDS = [0]
    DROP_REMAINDER = True # Drop remainder - dropping last batch if size < batch size
    EPOCHS = 35
    
    # Training monitoring
    PLOT_HISTORY = True
    
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    tf.random.set_seed(SEED)
    print('seeding done!!!')
    
seeding(CFG.SEED)

## Login to WANDB to log trainings

In [ ]:
import wandb
wandb.login(key="ed6c2fc334f7ae297c94626b3056901c86359321")

In [ ]:
wandb_config={
    "architecture": CFG.NETWORK_NAME,
    "input_shape": CFG.CNN_INPUT_SHAPE,
    "epochs": CFG.EPOCHS,
    "batch_size": CFG.BATCH_SIZE,
    "seed": CFG.SEED,
    "use_small_sample": False, 
}

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="flowers-augmentation",

    # track hyperparameters and run metadata with wandb.config
    config=wandb_config
)

wandb.log({
        "Augment": CFG.AUGMENT,
        "Traditional Prob": CFG.TRADITIONAL_AUG_PROB,
        "MixUp Prob": CFG.MIXUP_PROB,
        "MixUp Alpha": CFG.MIXUP_ALPHA,
        "CutMix Prob": CFG.CUTMIX_PROB,
        "CutMix Alpha": CFG.CUTMIX_ALPHA,
    })

## Load datasets

### Loading whole dataset

#### Extract all images from source dataset

In [ ]:
import tarfile 

if not os.path.isdir('/kaggle/working/flowers-102-source-extracted'):
    with tarfile.open('/kaggle/input/oxford-flowers-102-source/102flowers.tgz') as f: 
        f.extractall('/kaggle/working/flowers-102-source-extracted') 

#### Load whole dataset

In [ ]:
from scipy.io import loadmat

labels_mine = loadmat('/kaggle/input/oxford-flowers-102-source/imagelabels.mat')["labels"][0]

DIR_PATH = '/kaggle/working/flowers-102-source-extracted/jpg'

df_all = pd.DataFrame(columns=["filepath", "target", "targetName"])
idx = 0
for image, label in zip(sorted(os.listdir(DIR_PATH)), labels_mine):
    new_row = pd.DataFrame({"filepath": os.path.join(DIR_PATH, image),
                            "target": label,
                           "targetName": CFG.CATEGORIES[label]}, 
                               index=[idx])
    
    df_all = pd.concat([df_all, new_row], ignore_index=True)
    
    idx += 1
    
df_all["target"] = df_all["target"].astype(int)
print(df_all.head())
print(df_all.shape)

### Train-Test Split

#### Divide into Training and Test split

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming `df` is your DataFrame
# Splitting the DataFrame into train and test sets, stratified by the 'target' column
train_df, test_df = train_test_split(
    df_all, 
    test_size=0.2,  # 20% for testing, 80% for training
    stratify=df_all['target'],  # Stratify by the 'target' column to maintain class distribution
    random_state=CFG.SEED  # For reproducibility
)

#### Divide Training set into Folds

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=CFG.NUM_SPLITS, shuffle=True, random_state=CFG.SEED)

df = train_df.reset_index(drop=True)

df["fold"] = -1
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['target'])):
    df.loc[val_idx, 'fold'] = fold
    
# Apply the conversion logic
df['split'] = df['fold'].apply(lambda x: 'val' if x == 0 else 'train')

# Drop the original 'fold' column and rename it to 'split'
df = df.drop('fold', axis=1)

# Display or save the updated DataFrame
print(df)

## Visualize 10 random images

In [ ]:
def display_10_images(dataset):

    images = dataset.sample(10, random_state=CFG.SEED)

    plt.figure(figsize=(15, 6))  # Increase figure size for better visibility
    i = 0
    for _, image in images.iterrows():
        plt.subplot(2, 5, i + 1)  # 2 rows, 5 columns, ith+1 subplot
        plt.imshow(Image.open(image.filepath))
        plt.title(f'Class: {image.targetName}')
        plt.axis('off')
        i += 1
    plt.show()


In [ ]:
display_10_images(df)

## Data Augmentation

### Traditional Augmentation

In [ ]:
!pip install tensorflow_addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

def random_rotate_image(image):
    # Generate a random angle for rotation. The angle is in radians.
    # Here, we rotate between -45 and 45 degrees, converted to radians.
    angle_rad = tf.random.uniform(shape=[], minval=-0.25*np.pi, maxval=0.25*np.pi)
    # Rotate the image
    rotated_image = tfa.image.rotate(image, angles=angle_rad, interpolation='NEAREST', fill_mode='reflect')
    return rotated_image

# Generats random float
def random_float(shape=[], minval=0.0, maxval=1.0):
    rnd = tf.random.uniform(shape=shape, minval=minval, maxval=maxval, dtype=tf.float32)
    return rnd

def traditional_image_aug(image):
#     # Flipping left right
#     image = tf.image.random_flip_left_right(image)
    
    # Brightness
    image = tf.image.random_brightness(image, max_delta=0.23)
    
    # Contrast
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    
    # Gaussian Filter
    image = tfa.image.gaussian_filter2d(image, filter_shape=(3, 3), sigma=1.0)
    
    # Rotate max 45 degrees
    image = random_rotate_image(image)
    
    # Add gaussian noise
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.03, dtype=tf.float32)
    image = image + noise 
    
    # Random Cropping 
    max_ratio = random_float(minval=0.8, maxval=1.0)
    new_height = int(CFG.CNN_INPUT_HEIGHT * max_ratio)
    new_width = int(CFG.CNN_INPUT_WIDTH * max_ratio)
    image = tf.image.random_crop(image, size=[new_height, new_width, CFG.CNN_INPUT_CHANNELS])
    image = tf.image.resize(image, [CFG.CNN_INPUT_HEIGHT, CFG.CNN_INPUT_WIDTH])

    return image

def traditional_image_augmenter(with_labels=True):
    def augment(image):
        if random_float() <= CFG.TRADITIONAL_AUG_PROB:
            image = traditional_image_aug(image)
        return image

    def augment_with_labels(image, label):
        return augment(image), label

    return augment_with_labels if with_labels else augment

### Advanced Augmentation

#### MixUp

In [ ]:
def mixup_image_aug(images, labels, alpha=CFG.MIXUP_ALPHA):
    
    if random_float() > CFG.MIXUP_PROB:
        return images, labels
    
    image_shape = tf.shape(images)
    label_shape = tf.shape(labels)

    beta = tfp.distributions.Beta(alpha, alpha)
    lam = beta.sample(1)[0]

    images = lam * images + (1 - lam) * tf.roll(images, shift=1, axis=0)
    labels = lam * labels + (1 - lam) * tf.roll(labels, shift=1, axis=0)

    images = tf.reshape(images, image_shape)
    labels = tf.reshape(labels, label_shape)
    
    return images, labels

#### CutMix

In [ ]:

import tensorflow as tf



def create_cutmix_mask(bbx1, bby1, bbx2, bby2, height, width, channels, batch_size):
    # Create a grid of coordinates (height x width)
    x_coords = tf.range(width)
    y_coords = tf.range(height)
    Y, X = tf.meshgrid(y_coords, x_coords)

    # Reshape the bounding box coordinates to make them broadcastable over the batch size
    bbx1 = tf.reshape(bbx1, [batch_size, 1, 1])
    bby1 = tf.reshape(bby1, [batch_size, 1, 1])
    bbx2 = tf.reshape(bbx2, [batch_size, 1, 1])
    bby2 = tf.reshape(bby2, [batch_size, 1, 1])

    # Create the mask by comparing the coordinates
    mask = (X >= bbx1) & (X < bbx2) & (Y >= bby1) & (Y < bby2)
    mask = tf.cast(mask, tf.float32)  # Convert the mask to float32
    
    # Calculate ratio 
    patch_area = tf.reduce_sum(mask, axis=[1, 2])  # Sum over height, width, and channels dimensions
    total_area = height * width
    ratio = patch_area / tf.cast(total_area, tf.float32)
    ratio = tf.expand_dims(ratio, -1)
    ratio = tf.tile(ratio, [1, 102])
    # Add the channel dimension
    mask = tf.expand_dims(mask, -1)
    # Tile the mask across the channel dimension
    mask = tf.tile(mask, [1, 1, 1, channels])
    
    
    return mask, ratio


def cutmix(images, labels, probability=0.5, alpha=1.0):
    # Only apply CutMix with the given probability
    if random_float() > probability:
        return images, labels

    # Assume images is a 4D tensor of shape [batch_size, height, width, channels]
    shape = tf.shape(images)
    
    batch_size = shape[0]
    height = shape[1]
    width = shape[2]
    channels = shape[3]
    
    # Sample lambda and calculate patch dimensions
    beta = tfp.distributions.Beta(alpha, alpha)
    lambda_val = beta.sample(1)
#     beta_distribution = beta(alpha, alpha)
#     lambda_val = beta_distribution.rvs(size=1)[0]


    cut_rat = tf.sqrt(1. - lambda_val)
    
    cut_w = tf.cast(width, tf.float32) * cut_rat
    cut_w = tf.cast(cut_w, tf.int32)  # Now, cut_w is an int32 tensor.
    
    cut_h = tf.cast(height, tf.float32) * cut_rat
    cut_h = tf.cast(cut_h, tf.int32)  # Now, cut_h is an int32 tensor.
    
    # Uniformly sample the center of the patch
    cx = tf.random.uniform([batch_size], minval=0, maxval=width, dtype=tf.int32)
    cy = tf.random.uniform([batch_size], minval=0, maxval=height, dtype=tf.int32)
    
    # Calculate the patch coordinates
    bbx1 = tf.clip_by_value(cx - cut_w // 2, 0, width)
    bby1 = tf.clip_by_value(cy - cut_h // 2, 0, height)
    bbx2 = tf.clip_by_value(cx + cut_w // 2, 0, width)
    bby2 = tf.clip_by_value(cy + cut_h // 2, 0, height)
    
#     # Create mask
    mask, ratio = create_cutmix_mask(bbx1, bby1, bbx2, bby2, height, width, channels, batch_size)
    indices = tf.random.shuffle(tf.range(batch_size))
    mixed_images = images * mask + tf.gather(images, indices) * (1 - mask)

    # Mix the labels
    mixed_labels = labels * ratio + tf.gather(labels, indices) * (1 - ratio)
#     mixed_labels = labels
    
    return mixed_images, mixed_labels


## Train model

### Define metrics, loss, optimizer

In [ ]:
import sklearn.metrics

def get_metrics():
    auc = tf.keras.metrics.AUC(curve='PR', name='auc', multi_label=False) # auc on prcision-recall curve
    acc = tf.keras.metrics.CategoricalAccuracy(name='acc')
    return [acc, auc]

def padded_cmap(y_true, y_pred, padding_factor=5):
    num_classes = y_true.shape[1]
    pad_rows = np.array([[1]*num_classes]*padding_factor)
    y_true = np.concatenate([y_true, pad_rows])
    y_pred = np.concatenate([y_pred, pad_rows])
    score = sklearn.metrics.average_precision_score(y_true, y_pred, average='macro',)
    return score

def get_loss():
    if CFG.loss=="CCE":
        loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=CFG.label_smoothing)
    else:
        raise ValueError("Loss not found")
    return loss
    
def get_optimizer():
    if CFG.optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=CFG.lr)
    else:
        raise ValueError("Optmizer not found")
    return opt

In [ ]:

from tensorflow.keras.applications.resnet50 import ResNet50

def build_model_resnet(CFG, compile_model=True):
    """
    Builds and returns a model based on the specified configuration.
    """
  
    DIM = (None, None)

    # Base model - Resnet50
    base = tf.keras.applications.ResNet50(
      include_top=False,
      weights="imagenet",
      input_shape=(*DIM, 3),
    )

    # Input layer 
    inp = tf.keras.layers.Input(shape=(*DIM, 3))

    # Input -> base 
    out = base(inp)

    # GAP Layer
    out = tf.keras.layers.GlobalAveragePooling2D()(out)

    # Final dense layer for classification
    out = tf.keras.layers.Dense(len(CFG.CATEGORIES), activation='softmax')(out)

    # Create the TensorFlow model 
    model = tf.keras.Model(inputs=inp, outputs=out)
    if compile_model:
        # Optimizer
        opt = get_optimizer()
        # Loss function
        loss = get_loss()
        # Evaluation metrics
        metrics = get_metrics()
        # Compile the model 
        model.compile(optimizer=opt,
                      loss=loss,
                      metrics=metrics)
    
    return model

In [ ]:
m = build_model_resnet(CFG)

m.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def build_flattened_model_resnet(CFG, compile_model=True):

    DIM = (None, None)
    # Load the base ResNet50 model without the top classification layers
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*DIM, 3))

    # Make sure all layers are set to trainable for fine-tuning
    for layer in base_model.layers:
        layer.trainable = True

    # Add custom layers on top of ResNet50
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(len(CFG.CATEGORIES), activation='softmax', name='output_layer')(x)  # Change the number of units and activation based on your task

    # Create the complete model
    model = Model(inputs=base_model.input, outputs=x)

    # Print the model summary
    
    if compile_model:
        # Optimizer
        opt = get_optimizer()
        # Loss function
        loss = get_loss()
        # Evaluation metrics
        metrics = get_metrics()
        # Compile the model 
        model.compile(optimizer=opt,
                      loss=loss,
                      metrics=metrics)
    
    return model

In [ ]:
m = build_flattened_model_resnet(CFG)

# m.summary()

In [ ]:
def build_flattened_model_efficientnet(CFG, compile_model=True):

    DIM = (None, None)
    # Load the base ResNet50 model without the top classification layers
    base_model = tf.keras.applications.EfficientNetB4(
      include_top=False,
      weights="imagenet",
      input_shape=(*DIM, 3),
    )

    # Make sure all layers are set to trainable for fine-tuning
    for layer in base_model.layers:
        layer.trainable = True

    # Add custom layers on top of ResNet50
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(len(CFG.CATEGORIES), activation='softmax', name='output_layer')(x)  # Change the number of units and activation based on your task

    # Create the complete model
    model = Model(inputs=base_model.input, outputs=x)

    # Print the model summary
    
    if compile_model:
        # Optimizer
        opt = get_optimizer()
        # Loss function
        loss = get_loss()
        # Evaluation metrics
        metrics = get_metrics()
        # Compile the model 
        model.compile(optimizer=opt,
                      loss=loss,
                      metrics=metrics)
    
    return model

In [ ]:
m = build_flattened_model_efficientnet(CFG)

# m.summary()

# Train model using cross validation

In [ ]:
def image_decoder(path, label=None, with_labels=True):
    def image_loader(path):
        if not os.path.exists(path):
            pass
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, CFG.CNN_INPUT_SIZE)
        image = image / 255.0
        
        image.set_shape([CFG.CNN_INPUT_HEIGHT, CFG.CNN_INPUT_WIDTH, CFG.CNN_INPUT_CHANNELS])
        
        return image
    
    def target_loader(target):
#         target = tf.reshape(target, [1])
        target = tf.cast(tf.one_hot(target, depth=len(CFG.CATEGORIES)), dtype=tf.float32)
        return target #tf.reshape(target, [len(CFG.CATEGORIES)])

    def decode(path):
        image = image_loader(path)
        return image

    def decode_with_labels(path, label):
        label = target_loader(label)
        return decode(path), label
    if type(path) == str:
        return decode_with_labels(path, label) if with_labels else decode(path)
    return decode_with_labels(path.numpy(), label.numpy()) if with_labels else decode(path.numpy())
    

In [ ]:
def build_dataset(paths, labels=None, batch_size=CFG.BATCH_SIZE, target_size=CFG.CNN_INPUT_SIZE,
                  image_decoder_fn=None, traditional_augment_fn=None,
                  spec_decode_fn=None, mixup_image_augment_fn=None,
                  cache=True, cache_dir="",drop_remainder=False,
                  augment=True, repeat=True, shuffle=100):
    """
    Creates a TensorFlow dataset from the given paths and labels.

    Args:
        paths (list): A list of file paths to the audio files.
        labels (list): A list of corresponding labels for the audio files.
        batch_size (int): Batch size for the created dataset.
        target_size (list): A list of target image size for the spectrograms.
        audio_decode_fn (function): A function to decode the audio file.
        audio_augment_fn (function): A function to augment the audio file.
        spec_decode_fn (function): A function to decode the spectrogram.
        spec_augment_fn (function): A function to augment the spectrogram.
        cache (bool): Whether to cache the dataset or not.
        cache_dir (str): Directory path to cache the dataset.
        drop_remainder (bool): Whether to drop the last batch if it is smaller than batch_size.
        augment (bool): Whether to augment the dataset or not.
        repeat (bool): Whether to repeat the dataset or not.
        shuffle (int): Number of elements from the dataset to buffer for shuffling.

    Returns:
        ds (tf.data.Dataset): A TensorFlow dataset.
    """

    if traditional_augment_fn is None:
        traditional_augment_fn = traditional_image_augmenter(
            labels is not None)
        

    AUTO = tf.data.experimental.AUTOTUNE


    slices = paths if labels is None else (paths, labels)
    ds = tf.data.Dataset.from_tensor_slices(slices)

    if labels is None:
        ds = ds.map(lambda x: tf.py_function(image_decoder, [x], [tf.float32]), num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda x, y: tf.py_function(image_decoder, [x, y], [tf.float32, tf.float32]), num_parallel_calls=AUTO)
    

#     ds = ds.cache(cache_dir) if cache else ds

    ds = ds.repeat() if repeat else ds

#     opt = tf.data.Options()

    if shuffle:
        ds = ds.shuffle(shuffle, seed=CFG.SEED)
#         opt.experimental_deterministic = False



    ds = ds.map(traditional_augment_fn, num_parallel_calls=AUTO) if augment else ds

    ds = ds.batch(batch_size, drop_remainder=CFG.DROP_REMAINDER)

    ds = ds.map(mixup_image_aug, num_parallel_calls=AUTO) if augment else ds # if (augment and labels is not None) else ds
    
    ds = ds.map(lambda images, labels: cutmix(images, labels, probability=CFG.CUTMIX_PROB, alpha=CFG.CUTMIX_ALPHA), num_parallel_calls=AUTO) if augment else ds

    ds = ds.prefetch(AUTO)
    return ds

In [ ]:
def plot_history(history):
    """Plot trainign history, credit: @cdeotte"""
    epochs = len(history.history['auc'])
    plt.figure(figsize=(15,5))
    plt.plot(np.arange(epochs),history.history['auc'],'-o',label='Train AUC',color='#ff7f0e')
    plt.plot(np.arange(epochs),history.history['val_auc'],'-o',label='Val AUC',color='#1f77b4')
    x = np.argmax( history.history['val_auc'] ); y = np.max( history.history['val_auc'] )
    xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.2f'%y,size=14)
    plt.ylabel('AUC (PR)',size=14); plt.xlabel('Epoch',size=14)
    plt.legend(loc=2)
    plt2 = plt.gca().twinx()
    plt2.plot(np.arange(epochs),history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
    plt2.plot(np.arange(epochs),history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
    x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
    ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
    plt.ylabel('Loss',size=14)
    plt.title('Fold %i - Training Plot'%(fold+1),size=18)
    plt.legend(loc=3)
    plt.show()

## WANDB Logging

In [ ]:
!nvidia-smi

In [ ]:
from wandb.keras import WandbCallback


def train_val_test_fit(model, df, CFG, model_name="Model"):
    # Split dataset with cv filter
    train_df = df.query("split == 'train'").reset_index(drop=True) 
    valid_df = df.query("split=='val'").reset_index(drop=True) 

    # Get file paths and labels
    train_paths = train_df.filepath.values; train_labels = train_df.target.values
    valid_paths = valid_df.filepath.values; valid_labels = valid_df.target.values


    # Shuffle the file paths and labels
    index = np.arange(len(train_paths))
    np.random.shuffle(index)
    train_paths  = train_paths[index]
    train_labels = train_labels[index]

    # Compute the number of training and validation samples
    num_train = len(train_paths); num_valid = len(valid_paths)
        
    # # Build the training and validation datasets
    cache=True
    train_ds = build_dataset(train_paths, train_labels, 
                              batch_size=CFG.BATCH_SIZE, cache=cache, shuffle=True,
                            augment=CFG.AUGMENT, drop_remainder=CFG.DROP_REMAINDER)
    valid_ds = build_dataset(valid_paths, valid_labels,
                              batch_size=CFG.BATCH_SIZE, cache=cache, shuffle=False,
                              augment=False, repeat=False, drop_remainder=CFG.DROP_REMAINDER)


    # # Print information about the training
    print('#### Image Size: (%i, %i) | Model: %s | Batch Size: %i '%
          (*CFG.CNN_INPUT_SIZE, model_name, CFG.BATCH_SIZE))
    print('#### Num Train: {:,} | Num Valid: {:,}'.format(len(train_paths), len(valid_paths)))

    # # Callbacks
    sv = tf.keras.callbacks.ModelCheckpoint(
        'training_save.keras', monitor='val_acc', verbose=0, save_best_only=True,
        save_weights_only=False, mode='max', save_freq='epoch')
    callbacks = [sv,  
                 WandbCallback(generator=valid_ds)] # get_lr_callback(CFG.batch_size)

    # # Training
    print('# Training')
    history = model.fit(
        train_ds, 
        epochs=CFG.EPOCHS, 
        callbacks=callbacks, 
        steps_per_epoch=len(train_paths)//CFG.BATCH_SIZE,
        validation_data=valid_ds, 
        # verbose=CFG.verbose,
    )
    
    model = tf.keras.models.load_model('/kaggle/working/training_save.keras')
    
    return model, history

In [ ]:
 # # Clear the session, build and train the model
K.clear_session()
# model = build_model_resnet(CFG) 
model = build_model_resnet(CFG)
model, history = train_val_test_fit(model, df, CFG, model_name="ResNet")


### Predict on Test Set

#### Prediction utils

In [ ]:
def predict_classes(model, paths):
    fake_labels = np.zeros(test_paths.shape, dtype=int)
    ds = build_dataset(paths, fake_labels,
                    batch_size=1, cache=False, shuffle=False,
                    augment=False, repeat=False, drop_remainder=False)

    preds = model.predict(ds)
    pred_labels = np.argmax(preds, axis=1)

    return pred_labels, preds

#### Predict and calculate score

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def log_test_metrics(test_labels, preds):
    accuracy = accuracy_score(test_labels, preds)
    precision = precision_score(test_labels, preds, average='weighted') 
    recall = recall_score(test_labels, preds, average='weighted') 
    f1 = f1_score(test_labels, preds, average='weighted')  


    wandb.log({
        "Test Accuracy": accuracy,
        "Test Precision": precision,
        "Test Recall": recall,
        "Test F1 Score": f1
    })

    print(f"Accuracy: {accuracy} Precision: {precision} Recall: {recall} F1: {f1}")

In [ ]:
test_paths = test_df.filepath.values; test_labels = test_df.target.values

preds, probs = predict_classes(model, test_paths)

log_test_metrics(test_labels, preds)

#### WANDB: Log confusion matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def create_top_10_confusion_matrix(true_labels, preds, class_names):
    # Assuming adjusted_labels and adjusted_preds are properly defined
    num_classes = len(class_names)  # Total number of classes
    class_labels = range(num_classes)  # Zero-based class labels
    
    adjusted_labels = test_labels - 1
    adjusted_preds = preds - 1
    adjusted_categories = {k-1: v for k, v in CFG.CATEGORIES.items()}

    # Create the full confusion matrix with adjusted labels
    full_cm = confusion_matrix(adjusted_labels, adjusted_preds, labels=class_labels)

    # Calculate misclassification counts (not including correct classifications)
    misclassification_counts = full_cm.sum(axis=1) - np.diag(full_cm)
    total_true_counts = np.bincount(adjusted_labels, minlength=num_classes)
    misclassification_rates = misclassification_counts / total_true_counts

    # Identify top 10 most frequently misclassified classes
    top_10_classes = np.argsort(-misclassification_rates)[:10]

    # Create reduced confusion matrix for top 10 classes
    top_10_cm = full_cm[top_10_classes, :][:, top_10_classes]

    # Calculate "Other" row and column properly
    total_predictions_sum = full_cm.sum(axis=0)[top_10_classes]  # All predictions sum
    total_true_sum = full_cm.sum(axis=1)[top_10_classes]  # All true labels sum
    other_row = total_predictions_sum - top_10_cm.sum(axis=0)
    other_column = total_true_sum - top_10_cm.sum(axis=1)

    # Append the "Other" value
    other_value = np.nan
    
    # Forming the 11x11 confusion matrix
    final_cm = np.vstack([
        np.hstack([top_10_cm, other_column.reshape(-1, 1)]),
        np.hstack([other_row, other_value])
    ])

    # Labels for plotting (adjust labels back to 1-102 for readability)
    labels = [f'{CFG.CATEGORIES[i+1]}' for i in top_10_classes] + ['Other']

    # Plotting
    plt.figure(figsize=(12, 10))
    sns.heatmap(final_cm, annot=True, cmap="Blues", fmt=".0f", xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix for Top 10 Most Frequently Misclassified Categories')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.savefig("confusion_matrix.png")
    plt.show()
    
    return final_cm

top_10_cm = create_top_10_confusion_matrix(test_labels, preds, CFG.CATEGORIES)
wandb.log({"Top 10 Confusion Matrix": wandb.Image("confusion_matrix.png")})

In [ ]:
def plot_batch(batch, row=3, col=3):
    """Plot one batch data"""
    if isinstance(batch, tuple) or isinstance(batch, list):
        imgs, tars = batch
    else:
        imgs = batch
        tars = None
    plt.figure(figsize=(col*2, row*2))
    
    for idx in range(row*col):
        ax = plt.subplot(row, col, idx+1)
        ax.imshow(imgs[idx])
        plt.axis('off')

        if tars is not None:
            label = tars[idx].numpy().argmax()
            if label == 0:
                label = 102
            name = CFG.CATEGORIES[label]
            plt.title(name)
    plt.tight_layout()
    plt.show()

In [ ]:
DESIRED_BATCH = 3
ds = build_dataset(df.filepath.tolist(), df.target.tolist(), augment=False, cache=False, shuffle=None)
ds = ds.take(100)
ds_iter = iter(ds)

for idx, (imgs, labels) in enumerate(ds_iter):
    if idx == DESIRED_BATCH:
        plot_batch((imgs, labels), 2, 5)
        break

In [ ]:
ds = build_dataset(df.filepath.tolist(), df.target.tolist(), augment=True, cache=False, shuffle=None)
ds = ds.take(100)
ds_iter = iter(ds)

for idx, (imgs, labels) in enumerate(ds_iter):
    if idx == DESIRED_BATCH:
        plot_batch((imgs, labels), 2, 5)
        break

In [ ]:
wandb.finish()

## Load model from wandb

In [ ]:
import wandb

wandb.login(key="ed6c2fc334f7ae297c94626b3056901c86359321")


In [ ]:
wandb.init(project='flowers-augmentation', 
           entity='kmotyka2000org', 
           id='igzrho7g', 
           resume='must')

In [ ]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project="flowers-augmentation",
)

In [ ]:
import tensorflow as tf

entity = "kmotyka2000org"
project = "flowers-augmentation"
artifact_name = "model-driven-haze-21"
epoch = "latest"

artifact = run.use_artifact(f'{entity}/{project}/{artifact_name}:{epoch}')

# Download the model
artifact_dir = artifact.download()

# Load the model
model = tf.keras.models.load_model(artifact_dir)

In [ ]:
model = tf.keras.models.load_model('/kaggle/input/no-aug-best/tensorflow2/no-aug-model/1/model-best.h5')

In [ ]:
wandb.finish()

### GradCAM

In [ ]:
!pip install tf_keras_vis
from tf_keras_vis.gradcam import Gradcam, GradcamPlusPlus
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
import cv2

In [ ]:
samples_20 = test_df.sample(20, random_state=CFG.SEED)

In [ ]:
x_df = samples_20.sample(1, random_state=CFG.SEED)

print(x_df)
score = CategoricalScore(x_df.target.values[0])
image = image_decoder(x_df.filepath.values[0], with_labels=False)

# Batch
batch_image = tf.expand_dims(image, 0)

pred = model.predict(batch_image)

print(f"Predicted: {np.argmax(pred, axis=1)[0]}, real: {x_df.target.values[0]}")

plt.imshow(image)
plt.title(CFG.CATEGORIES[x_df.target.values[0]])
plt.show()

In [ ]:
gradcam = GradcamPlusPlus(model, model_modifier=ReplaceToLinear(), clone=False) 

cam = gradcam(score, batch_image, penultimate_layer=-1)

# Since the image is a single input, we'll get the heatmap for the first image
heatmap = cam[0]

# Normalize the heatmap to [0, 1] range for better visualization
heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

# Visualize the heatmap
plt.imshow(heatmap)
plt.title('Grad-CAM Heatmap')
plt.colorbar()
plt.show()


# Visualize
heatmap = np.uint8(255 * heatmap)
image = np.uint8(255 * image)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Overlay the heatmap on the original image
superimposed_img = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)

# Display the original image, heatmap, and superimposed image
plt.figure(figsize=(5, 5))

plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.title('Grad-CAM Overlay')

plt.show()

In [ ]:
example = test_df[test_df["target"] == 73].sample(1)
plt.imshow(image_decoder(example.filepath.values[0], with_labels=False))
plt.title(CFG.CATEGORIES[example.target.values[0]])
plt.show()

In [ ]:
!mkdir /kaggle/working/gradcam

In [ ]:
for index, x_df in test_df.reset_index().iterrows():
    # Load image
    score = CategoricalScore(x_df.target if x_df.target != 102 else 0)
    image = image_decoder(x_df.filepath, with_labels=False)

    # Batch
    batch_image = tf.expand_dims(image, 0)
    
    # Predict
    pred = model.predict(batch_image)
    
    gradcam = Gradcam(model, model_modifier=ReplaceToLinear(), clone=False) 

    cam = gradcam(score, batch_image, penultimate_layer=-1)

    heatmap = cam[0]
    
    # Normalize
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())
    
    heatmap = np.uint8(255 * heatmap)
    image = np.uint8(255 * image)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Overlay the heatmap on the original image
    superimposed_img = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)

    # Display the original image, heatmap, and superimposed image
    plt.figure(figsize=(5, 5))

    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title(f'Grad-CAM: pred:: {np.argmax(pred, axis=1)[0]}, real: {x_df.target if x_df.target != 102 else 0}')
    plt.savefig(f'/kaggle/working/gradcam/GradCAM_{index}.png')
    plt.close()
    
#     break
!zip -r /kaggle/working/gradcam.zip /kaggle/working/gradcam


In [ ]:
for index, x_df in test_df.reset_index().iterrows():
    # Load image
    score = CategoricalScore(x_df.target if x_df.target != 102 else 0)
    image = image_decoder(x_df.filepath, with_labels=False)

    # Batch
    batch_image = tf.expand_dims(image, 0)
    
    # Predict
    pred = model.predict(batch_image)
    
    gradcam = GradcamPlusPlus(model, model_modifier=ReplaceToLinear(), clone=False) 

    cam = gradcam(score, batch_image, penultimate_layer=-1)

    heatmap = cam[0]
    
    # Normalize
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())
    
    heatmap = np.uint8(255 * heatmap)
    image = np.uint8(255 * image)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Overlay the heatmap on the original image
    superimposed_img = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)

    # Display the original image, heatmap, and superimposed image
    plt.figure(figsize=(5, 5))

    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title(f'Grad-CAM: pred:: {np.argmax(pred, axis=1)[0]}, real: {x_df.target if x_df.target != 102 else 0}')
    plt.savefig(f'/kaggle/working/gradcam_plusplus/GradCAM_{index}.png')
    plt.close()
    
#     break
!zip -r /kaggle/working/gradcam_plusplus.zip /kaggle/working/gradcam_plusplus


In [ ]:
!mkdir /kaggle/working/processed_images

In [ ]:
for index, x_df in test_df.reset_index().iterrows():


    image = image_decoder(x_df.filepath, with_labels=False)


    # Display the original image, heatmap, and superimposed image
    plt.figure(figsize=(5, 5))

    plt.imshow(image)
    plt.title(f'Class: {CFG.CATEGORIES[x_df.target]}')
    plt.savefig(f'/kaggle/working/processed_images/image_{index}.png')
    plt.close()
    
!zip -r /kaggle/working/processed_images.zip /kaggle/working/processed_images
